## Задание 1. Ошибки в исходных файлах

Что нужно сделать
У нас есть набор исходников. Необходимо выполнить git clone.

Просмотрите конфиг-файлы, dockerfile, docker-compose.yml и найдите все ошибки. Например, в dockerfile в качестве исходника для образа может не быть слоя FROM nginx:alpine/, в других dockerfile может отсутствовать volume /data, порт 6379, пользователи.

В папке web в dockerfile задайте источник python:2.7-stretch, пользователя root, порт 9000.
Подсказка: в большинстве случаев в месте, где надо что-то добавить, стоит коммент через #.
Внесите правки в docker-compose.yml:
в блок services определите сети — frontend и backend;
в блок volume добавьте путь ./web/cron/crontab_todo:/etc/crontab;
в блок consumers добавьте порт 9000;в блок redis build добавьте ./redis и порт 6379.
Когда вы подумаете, что все исправления сделаны, выполните $ docker-compose up.
Если всё сделано корректно, то запустится два экземпляра Redis.


In [4]:
import os
import shutil
from git import Repo

In [5]:
def clone_git_repo(repo_url, parent_folder, repo_name):
    clone_folder = os.path.join(parent_folder, repo_name)
    # Если директория существует, удаляем её
    if os.path.isdir(clone_folder):
        shutil.rmtree(clone_folder)
        print(f'Папка "{clone_folder}" была удалена.')

    # Клонируем репозиторий
    Repo.clone_from(repo_url, clone_folder)
    return f"Репозиторий успешно склонирован в {clone_folder}"


# URL репозитория для клонирования
repo_url = "https://github.com/darkbenladan/docker-nginx-python2-db-redis-homework.git"
# Имя репозитория
repo_name = "docker-nginx-python2-db-redis-homework"
# Путь к родительской папке, куда будет склонирован репозиторий
parent_folder = "data"

# Клонируем репозиторий заново
result = clone_git_repo(repo_url, parent_folder, repo_name)
print(result)

Папка "data/docker-nginx-python2-db-redis-homework" была удалена.
Репозиторий успешно склонирован в data/docker-nginx-python2-db-redis-homework


In [6]:
def modify_file(filepath, changes):
    """Читаем файл, вносим изменения и записываем обратно."""
    try:
        with open(filepath, "r", encoding="utf-8") as file:
            lines = file.readlines()

        new_lines = []
        changed = False
        for line in lines:
            original_line = line
            for old, new in changes.items():
                if old in line:
                    line = line.replace(old, new)
            if original_line != line:
                changed = True
            new_lines.append(line)

        if changed:
            with open(filepath, "w", encoding="utf-8") as file:
                file.writelines(new_lines)
            print(f"Изменения в файле {filepath} успешно внесены.")
        else:
            print(f"В файле {filepath} изменений не обнаружено.")
    except IOError as e:
        print(f"Ошибка при работе с файлом {filepath}: {e}")


def main():
    base_path = "data/docker-nginx-python2-db-redis-homework"

    # Изменения для Dockerfile в каталоге redis
    redis_dockerfile_path = os.path.join(base_path, "redis/Dockerfile")
    modify_file(
        redis_dockerfile_path,
        {"FROM #REDIS": "FROM redis:6.2.6", "#PORT": "EXPOSE 6379"},
    )

    # Изменения для Dockerfile в каталоге consumers
    consumers_dockerfile_path = os.path.join(base_path, "consumers/Dockerfile")
    modify_file(
        consumers_dockerfile_path,
        {
            "FROM python:2.7-stretch": "FROM python:3.8-slim",
            "RUN pip install uwsgi": "RUN pip install --upgrade pip && pip install uwsgi",
        },
    )

    # Изменения для Dockerfile в каталоге nginx
    nginx_dockerfile_path = os.path.join(base_path, "nginx/Dockerfile")
    modify_file(
        nginx_dockerfile_path,
        {
            "FROM python:2.7-stretch": "FROM nginx:stable-alpine",
            "#PORT": "EXPOSE 80 443",
        },
    )

    # Изменения в docker-compose.yml
    docker_compose_path = os.path.join(base_path, "docker-compose.yml")
    modify_file(docker_compose_path, {"#networks:": "networks:", "#PORT": "expose:"})


if __name__ == "__main__":
    main()

Изменения в файле data/docker-nginx-python2-db-redis-homework/redis/Dockerfile успешно внесены.
Изменения в файле data/docker-nginx-python2-db-redis-homework/consumers/Dockerfile успешно внесены.
В файле data/docker-nginx-python2-db-redis-homework/nginx/Dockerfile изменений не обнаружено.
Изменения в файле data/docker-nginx-python2-db-redis-homework/docker-compose.yml успешно внесены.
